<a href="https://colab.research.google.com/github/somosnlp/llm-arena/blob/main/Modismos_o_no.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
print(torch.cuda.is_available())  # Debe dar True


False


# Solicitar acceso al modelo en Hugging Face  (ver aquí cómo solicitar acceso: https://www.youtube.com/watch?v=c0PXr1Eazr8).
después de eso autentificarte con tu token

# Autenticarse con tu cuenta de Hugging Face
*Se abrirá un cuadro para que pegues tu token de acceso personal (se obtiene desde https://huggingface.co/settings/tokens).

In [ ]:
from huggingface_hub import login
login()


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# 📌 Configura tu modelo
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32, device_map="cpu"

)

# ✍️ Prompt personalizado
def create_prompt(sentence):
    return f"Does the following sentence contain an idiom? Answer 'Yes' or 'No'.\nSentence: {sentence}\nAnswer:"

# 🔍 Clasificador de modismos
def classify_idiom(sentence):
    prompt = create_prompt(sentence)
    print(f"▶️ Prompt enviado al modelo:\n{prompt}\n")  # Diagnóstico
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = output_text.lower().strip()
    return {
        "sentence": sentence,
        "answer": answer,
        "idiomatic": "yes" in answer
    }

# 🧪 Anotación en dataset de Hugging Face
def annotate_dataset(dataset_name="Conijeldres/dataset_preferencias_dpo_v0", column="question", n_samples=10):
    dataset = load_dataset(dataset_name, split="train")
    sentences = dataset.select(range(n_samples))[column]
    results = [classify_idiom(sent) for sent in sentences]
    return results

# ✅ Ejemplo de uso
if __name__ == "__main__":
    results = annotate_dataset(
        dataset_name="Conijeldres/dataset_preferencias_dpo_v0",  # cambia por el tuyo
        column="question",  # cambia por el nombre real de tu columna
        n_samples=20
    )

    for r in results:
        print(r)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-6887136c-04a9df5817a5e37442047979;ca40a194-6edf-4aa1-9847-76866930c570)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
from huggingface_hub import login
login()


In [ ]:
# 📦 Instalar e importar dependencias
!pip install transformers accelerate datasets --quiet

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
from google.colab import drive
import os

# 🚗 Montar Drive y definir ruta de salida
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/graficos_dpo/deteccion_modismos.csv"

# 📥 Cargar modelo optimizado
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# 🧠 Prompt con ejemplos
def create_prompt(sentence):
    ejemplos = (
        "Ejemplos:\n"
        "Oración: Se fue al chancho con la comida.\nRespuesta: Sí\n"
        "Oración: María compró pan en la esquina.\nRespuesta: No\n"
        "Oración: Le echó la foca a su jefe.\nRespuesta: Sí\n"
        "Oración: El perro duerme mucho.\nRespuesta: No\n\n"
    )
    return ejemplos + f"Ahora evalúa la siguiente oración:\nOración: {sentence}\nRespuesta:"

# 🔍 Clasificador de modismos
def classify_idiom(sentence):
    prompt = create_prompt(sentence)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True).lower().strip()
    respuesta = output_text.split("\n")[0].strip()
    return respuesta, respuesta.startswith("sí")

# 🧾 Cargar dataset desde Hugging Face
dataset = load_dataset("Conijeldres/dataset_preferencias_dpo_v0", split="train")

# ✨ Procesar una muestra o todo el dataset (ajusta n_samples)
n_samples = len(dataset)
results = []

for i in range(n_samples):
    row = dataset[i]
    for col in ["question", "chosen", "rejected"]:
        sentence = row[col]
        respuesta, es_idioma = classify_idiom(sentence)
        results.append({
            "country": row["country"],
            "type": col,
            "sentence": sentence,
            "respuesta_modelo": respuesta,
            "idiomatico": es_idioma
        })

# 💾 Guardar como CSV
df_resultados = pd.DataFrame(results)
df_resultados.to_csv(output_path, index=False)
print(f"✅ Archivo guardado en: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Guardar los resultados en un CSV

In [ ]:
import pandas as pd

df_resultados = pd.DataFrame(results)
df_resultados.to_csv("/content/drive/MyDrive/graficos_dpo/modismos_detectados.csv", index=False)
